In [1]:
!pip install ultralytics -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.1/899.1 kB 15.2 MB/s eta 0:00:00


In [2]:
!pip install kaggle -q

In [2]:
!git clone https://github.com/diaspeixoto/PDSI-E9.git

Cloning into 'PDSI-E9'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 15.45 MiB | 37.24 MiB/s, done.


In [3]:
!mk dir  -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!cd /content/
!kaggle datasets download rajdalsaniya/pothole-detection-dataset
!unzip -q pothole-detection-dataset.zip -d pothole-detection-dataset

/bin/bash: line 1: mk: command not found
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rajdalsaniya/pothole-detection-dataset
License(s): CC0-1.0
 91% 214M/235M [00:02<00:00, 100MB/s]
100% 235M/235M [00:02<00:00, 111MB/s]


In [4]:
import os
import glob
import cv2
import yaml
import numpy as np
import pandas as pd
from IPython.display import Image, display
from ultralytics import YOLO
from PIL import Image as PILImage
import torch
from google.colab import files
import shutil

print(f"Setup completo. Usando torch {torch.__version__} "
      f"({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Setup completo. Usando torch 2.5.1+cpu (CPU)


In [ ]:
# Configurar o arquivo de configuração do dataset
dataset_config = '''
train: ..//content/pothole-detection-dataset/train/images
val: ../pothole-detection-dataset/valid/images

nc: 1
names: ['pothole']
'''
with open('data.yaml', 'w') as f:
    f.write(dataset_config)


In [ ]:
# Carregar o arquivo de configuração YAML
with open('data.yaml', 'r') as f:
    config_data = yaml.safe_load(f)
print("Configuração carregada:", config_data)


Configuração carregada: {'train': '..//content/pothole-detection-dataset/train/images', 'val': '../pothole-detection-dataset/valid/images', 'nc': 1, 'names': ['pothole']}


In [ ]:
# Verificar a forma de uma imagem do dataset
image_list = glob.glob("/content/pothole-detection-dataset/train/images/*")
print(f"Número de imagens no dataset de treino: {len(image_list)}")



Número de imagens no dataset de treino: 2067


In [ ]:
if image_list:
    img = cv2.imread(image_list[0])
    img_shape = img.shape
    print(f"Dimensões da imagem: {img_shape}")
else:
    raise FileNotFoundError("Nenhuma imagem encontrada no dataset de treino.")

Dimensões da imagem: (640, 640, 3)


In [ ]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

model.train(
    data='data.yaml',
    epochs=15,
    imgsz=640,
    workers=4
)

Transferred 355/355 items from pretrained weights
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train2, exist_ok=False, pretrained=yolov8n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

train: Scanning /content/pothole-detection-dataset/train/labels.cache... 2067 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2067/2067 [00:00<?, ?it/s]
val: Scanning /content/pothole-detection-dataset/valid/labels.cache... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]

Plotting labels to runs/detect/train2/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


In [5]:
trained_model_path = 'runs/detect/train/weights/best.pt'
files.download(trained_model_path)

FileNotFoundError: Cannot find file: runs/detect/train/weights/best.pt

In [ ]:
# Exportar o modelo treinado
model.export()


In [7]:
# Fazer previsões em imagens de validação
validation_images = [
    "/kaggle/input/pothole-detection-dataset/valid/images/12_jpg.rf.b7c796cd5cf305af4937c9b3d38a4434.jpg",
    "/kaggle/input/pothole-detection-dataset/valid/images/552_jpg.rf.47c9daa4474beee1ebaaa8979a76abae.jpg"
]
results = model(validation_images, stream=False)


NameError: name 'model' is not defined